"Players" dataset observations:
- 196 observations
- 7 variables: experience (beginner, amateur, regular, pro, veteran), newsletter subscription (true/false), email, hours played, name, gender, and age
- experience, gender, newsletter subscription = categorical variables
- age, hours played = quantitative variables
- name, email = qualitative (not used for data analysis)
- potential errors: subjective ranking system for experience, and not enough specificity in 'hours played' variable: does it mean hours played per session, per numerous sessions, per month, per lifetime, etc.

"Sessions" dataset observations:
- 1535 observations
- 5 variables: email, start time (D/M/Y H:M format), end time (same format as prior), original start time (as milliseconds since 1 January 1970, a unix timestamp), and original end time (same format as prior)
- original start time, original end time, start time, end time = quantitative variables
- email = qualitative (used as an identifier only)
- potential errors: logs every session, including very short ones, which causes original start and end variables to be the same, original start and end variables are measured in milliseconds since 1 January 1970, making them difficult to interpret and analyze

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

sessions_cleaned <- sessions |> mutate(email = as_factor(hashedEmail)) |> select(-hashedEmail)

players_sorted <- players |> mutate(email = as_factor(hashedEmail))
sessions_named <- sessions_cleaned |> left_join(players_sorted, by = "email") |> select(-hashedEmail)

data_final <- sessions_named |> mutate(start = dmy_hm(start_time), end = dmy_hm(end_time), start_date = as_date(start), 
start_time = format(start, "%H:%M"), end_date = as_date(end_time), end_time = format(end, "%H:%M")) |>
select(-start, end, email)

#These lines aren't part of wrangling, they're just for computing summary statistics
mean_age <- players |> summarise(mean_age = mean(Age, na.rm = TRUE))
mean_time <- players |> summarise(mean_time = mean(played_hours, na.rm = TRUE))
mean_values <- tibble(mean_time, mean_age)
mean_values

#visualizations

plot_1 <- data_final |> ggplot(aes(x = gender, y = subscribe)) + geom_bar(stat = "identity", position = "dodge") + xlab("Gender") +
ylab("Time Played (hours)")
plot_1

Questions:
- Broad: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
- Specific: Can age, gender, and experience level predict whether or not someone will subscribe to a game-related newsletters?